# Example Usage

In [1]:
import pathlib

import pandas as pd

## Create Tables

In [2]:
# Import the module that allows us to create tables
import generate_tables as gt

### Creating Estimates Tables (generate_tables.EstimatesTables)

In [3]:
# Creating consolidated files using the vintage 2020_06 at the region and jurisdiction level
# "_ =" to supress output
_ = gt.EstimatesTables().consolidate("2020_06", 
        geo_list=["region", "jurisdiction"], 
        save=True,
        save_folder=pathlib.Path("./data/raw_data/"))

In [4]:
# Creating individual files using the vintage 2020_06 at the region and jurisdiction level for the
# Estimates tables household_income and age_ethnicity
_ = gt.EstimatesTables().individual("2020_06",
        geo_list=["region", "jurisdiction"],
        est_table_list=["household_income", "age_ethnicity"],
        save=True,
        save_folder=pathlib.Path("./data/raw_data/"))

### Creating DOF Data Files (generate_tables.CA_DOF)

In [5]:
# Creating DOF Data file.
# NOTE: See class documentation for some manual steps you need to do before running this code.
# Using the default raw_data and save locations
_ = gt.CA_DOF().get_CA_DOF_data(
    years=range(2010, 2022),
    geo_list=["region", "jurisdiction"])

### Creating Diff Files (generate_tables.DiffFiles)

In [6]:
# First create consolidated and individual files using the same parameters as before but with 
# a vintage of 2021_01
_ = gt.EstimatesTables().consolidate("2021_01", 
        geo_list=["region", "jurisdiction"], 
        save=True,
        save_folder=pathlib.Path("./data/raw_data/"))
_ = gt.EstimatesTables().individual("2021_01",
        geo_list=["region", "jurisdiction"],
        est_table_list=["household_income", "age_ethnicity"],
        save=True,
        save_folder=pathlib.Path("./data/raw_data/"))

In [7]:
# Create diff files 
_ = gt.DiffFiles().create_diff_tables("2020_06", "2021_01", 
        geo_list=["region", "jurisdiction"],
        est_table_list=["consolidated", "household_income", "age_ethnicity"],
        raw_data_folder=pathlib.Path("./data/raw_data/"),
        save_folder=pathlib.Path("./data/diff/"))

## Run Checks

In [8]:
# Import the module that allows us to run checks
import perform_checks as pc

### Run Check 1

In [9]:
# TODO

### Run Check 2

In [10]:
# TODO

### Run Check 3

In [11]:
# TODO

### Run Check 4

In [12]:
# TODO

### Run Check 5

In [13]:
# N/A, done in Power BI

### Run Check 6

In [14]:
# TODO

### Run Check 7

In [15]:
# TODO